In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import os

In [3]:
os.getcwd()

'c:\\Users\\swc08\\Desktop\\대학\\비어플\\프로젝트1'

In [2]:
non_fcf = pd.read_csv('seoul_nonfranchises_cafe_open.csv')
non_fcf

,사업장명,상세영업상태명,소재지면적,지번주소,도로명주소,좌표정보(X),좌표정보(Y)
0,썬푸드 딜리버리 관악점,영업,6.05,서울특별시 관악구 봉천동 1562-17,"서울특별시 관악구 쑥고개로 78, 1층 (봉천동)",195070.4745,441897.5178
1,보통공간,영업,32.00,서울특별시 마포구 연남동 224-14,"서울특별시 마포구 동교로52길 4-3, B01호 (연남동)",193461.2134,451300.1776
2,칼도 에스프레소바,영업,23.80,서울특별시 강서구 마곡동 757-5 프라이빗타워타워1차,"서울특별시 강서구 마곡중앙로 165, 프라이빗타워타워1차 1층 114호 (마곡동)",184708.9163,451822.7570
3,?커피,영업,33.00,서울특별시 성동구 용답동 73-2,"서울특별시 성동구 천호대로 292, 1층 (용답동)",204609.2990,451552.7243
4,팜스365,영업,89.25,서울특별시 마포구 성산동 515 월드컵주경기장,"서울특별시 마포구 월드컵로 240, 홈플러스 월드컵점 2층 x9.5,yuw호 (성산동)",191010.3810,451972.1540
...,...,...,...,...,...,...,...
11825,세마 카페 플러스(SeMA cafe +),영업,58.00,서울특별시 종로구 평창동 148-7 평창동미술문화복합공간B 1층,"서울특별시 종로구 평창문화로 101, 평창동미술문화복합공간B 1층 (평창동)",197470.7835,456097.9941
11826,오오디커피랩(Oodi coffee Lab),영업,25.29,서울특별시 구로구 개봉동 119-4,"서울특별시 구로구 고척로27길 38, 1층 (개봉동)",186394.0339,444612.0940
11827,디에스랩(DS_LAB),영업,78.00,"서울특별시 서초구 방배동 851-6 태창빌딩 1층 가동 103,1호","서울특별시 서초구 방배로 174, 태창빌딩 가동 1층 103,104호 (방배동)",199317.6906,443010.4062
11828,어니스트[:EARNEST],영업,66.00,서울특별시 강서구 등촌동 644-12 성명빌딩,"서울특별시 강서구 공항대로55길 33, 성명빌딩 1층 (등촌동)",187670.7003,450273.9989


In [3]:
non_fcf.drop(['좌표정보(X)','좌표정보(Y)'], axis=1, inplace=True)

In [4]:
non_fcf[['도로명주소', '상세주소']] = non_fcf['도로명주소'].str.split(',', n=1, expand=True)

In [6]:
import requests
import pandas as pd

# 네이버 API 클라이언트 정보
client_id = "3fmd7xia0h"
client_secret = "uy5gOT87pjBgIeAmXCH2dpXVBKK090HceaprqeKL"

# API 요청 함수 정의
def get_coordinates(address):
    endpoint = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode"
    url = f"{endpoint}?query={address}"

    # API 요청 헤더
    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret,
    }

    # API 호출 및 좌표 추출
    res = requests.get(url, headers=headers)
    if res.status_code == 200:
        data = res.json()
        if data['status'] == 'OK' and data['meta']['totalCount'] > 0:
            x = data['addresses'][0]['x']
            y = data['addresses'][0]['y']
            return pd.Series([x, y])
    return pd.Series([None, None])


In [7]:
# 주소 열에 대해 좌표를 적용하여 x, y 좌표 열 추가
non_fcf[['x', 'y']] = non_fcf['도로명주소'].apply(get_coordinates)

# 결과 확인
non_fcf.head()

,사업장명,상세영업상태명,소재지면적,지번주소,도로명주소,상세주소,x,y
0,썬푸드 딜리버리 관악점,영업,6.05,서울특별시 관악구 봉천동 1562-17,서울특별시 관악구 쑥고개로 78,1층 (봉천동),126.9450757,37.4792827
1,보통공간,영업,32.00,서울특별시 마포구 연남동 224-14,서울특별시 마포구 동교로52길 4-3,B01호 (연남동),126.9267885,37.5639580
2,칼도 에스프레소바,영업,23.80,서울특별시 강서구 마곡동 757-5 프라이빗타워타워1차,서울특별시 강서구 마곡중앙로 165,프라이빗타워타워1차 1층 114호 (마곡동),126.8277244,37.5685855
3,?커피,영업,33.00,서울특별시 성동구 용답동 73-2,서울특별시 성동구 천호대로 292,1층 (용답동),127.0529625,37.5662533
4,팜스365,영업,89.25,서울특별시 마포구 성산동 515 월드컵주경기장,서울특별시 마포구 월드컵로 240,"홈플러스 월드컵점 2층 x9.5,yuw호 (성산동)",126.8972730,37.5682885


In [10]:
non_fcf[non_fcf['x'].isna()]

,사업장명,상세영업상태명,소재지면적,지번주소,도로명주소,상세주소,x,y
137,후식커피,영업,22.00,서울특별시 동작구 사당동 736-1 총신대입구(이수)역4호선,서울특별시 동작구 동작대로 지하 117,총신대입구(이수)역4호선 지하1층 26호 (사당동),None,None
319,CAFE 폭포(카페 폭포),영업,358.70,서울특별시 서대문구 홍은동 429,NaN,NaN,None,None
843,명인만두 강남역지하상가점,영업,26.40,서울특별시 강남구 역삼동 858 강남역,서울특별시 강남구 강남대로 지하 396,강남역 지하1층 B29호 (역삼동),None,None
1080,뚜레쥬르 다올,영업,93.19,서울특별시 서초구 내곡동 1-1459 4층,NaN,NaN,None,None
1130,폼드팡,영업,65.36,서울특별시 강남구 청담동 77-76 청담역,서울특별시 강남구 학동로 지하 508,청담역 지하1층 729-306호 (청담동),None,None
...,...,...,...,...,...,...,...,...
11323,(주)하우 잠실역점,영업,12.00,서울특별시 송파구 신천동 8 잠실역2호선,서울특별시 송파구 올림픽로 지하 265,잠실역 지하1층 216-125호 (신천동),None,None
11334,브릭스콘,영업,20.90,서울특별시 송파구 신천동 8 잠실역2호선,서울특별시 송파구 올림픽로 지하 265,잠실역2호선(112호 합산) 지하층 216-111호 (신천동),None,None
11353,커피온리 잠실점,영업,10.38,서울특별시 송파구 신천동 8 잠실역2호선,서울특별시 송파구 올림픽로 지하 265,잠실역2호선 지하1층 216-114호 (신천동),None,None
11429,쁘띠아르브르 잠실한신점,영업,12.24,서울특별시 송파구 신천동 11-9번지,서울특별시 송파구 신천로6길 11,"127호 (신천동, 한신잠실코아)",None,None


In [11]:
non_fcf.to_csv('non_fcf_xy.csv', index=False)

In [12]:
pd.read_csv('non_fcf_xy.csv')

,사업장명,상세영업상태명,소재지면적,지번주소,도로명주소,상세주소,x,y
0,썬푸드 딜리버리 관악점,영업,6.05,서울특별시 관악구 봉천동 1562-17,서울특별시 관악구 쑥고개로 78,1층 (봉천동),126.945076,37.479283
1,보통공간,영업,32.00,서울특별시 마포구 연남동 224-14,서울특별시 마포구 동교로52길 4-3,B01호 (연남동),126.926789,37.563958
2,칼도 에스프레소바,영업,23.80,서울특별시 강서구 마곡동 757-5 프라이빗타워타워1차,서울특별시 강서구 마곡중앙로 165,프라이빗타워타워1차 1층 114호 (마곡동),126.827724,37.568585
3,?커피,영업,33.00,서울특별시 성동구 용답동 73-2,서울특별시 성동구 천호대로 292,1층 (용답동),127.052963,37.566253
4,팜스365,영업,89.25,서울특별시 마포구 성산동 515 월드컵주경기장,서울특별시 마포구 월드컵로 240,"홈플러스 월드컵점 2층 x9.5,yuw호 (성산동)",126.897273,37.568289
...,...,...,...,...,...,...,...,...
11825,세마 카페 플러스(SeMA cafe +),영업,58.00,서울특별시 종로구 평창동 148-7 평창동미술문화복합공간B 1층,서울특별시 종로구 평창문화로 101,평창동미술문화복합공간B 1층 (평창동),126.971902,37.607801
11826,오오디커피랩(Oodi coffee Lab),영업,25.29,서울특별시 구로구 개봉동 119-4,서울특별시 구로구 고척로27길 38,1층 (개봉동),126.846949,37.503599
11827,디에스랩(DS_LAB),영업,78.00,"서울특별시 서초구 방배동 851-6 태창빌딩 1층 가동 103,1호",서울특별시 서초구 방배로 174,"태창빌딩 가동 1층 103,104호 (방배동)",126.993061,37.489183
11828,어니스트[:EARNEST],영업,66.00,서울특별시 강서구 등촌동 644-12 성명빌딩,서울특별시 강서구 공항대로55길 33,성명빌딩 1층 (등촌동),126.861304,37.554652
